In [1]:
!pip install transformers qwen_vl_utils accelerate>=0.26.0 PEFT -U

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.1 MB/s eta 0:00:00


In [3]:
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
import torch
import os
from qwen_vl_utils import process_vision_info



model_name = "NAMAA-Space/Qari-OCR-0.1-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
                model_name,
                torch_dtype="auto",
                device_map="auto"
            )
processor = AutoProcessor.from_pretrained(model_name)
max_tokens = 2000

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.67k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
from PIL import Image
import requests
url = "https://paperspacearabicnlp.nyc3.digitaloceanspaces.com/samples/2.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [29]:
# src="/content/WhatsApp Image 2025-03-05 at 00.06.19_8e0f6443.jpg"
src= "/content/Screenshot 2025-03-05 012315.png"

# prompt ="""Extract the following details from the Egyptian National ID card in the image:

# Full Name (as written on the card)
# National ID Number (a 14-digit number with NO spaces, dashes (-), slashes (/), or extra characters)
# Date of Birth (DD-MM-YYYY format, must NOT be the same as the ID number)
# Address (as written on the card)
# Return only the extracted details in valid JSON format:

# json
# Copy
# Edit
# {
#   "name": "Extracted Name",
#   "id_number": "EXACT 14-DIGIT ID NUMBER FROM IMAGE",
#   "date_of_birth": "EXTRACTED DATE",
#   "address": "Extracted Address"
# }
# ⚠️ Important:

# The ID number must be directly extracted from the image (do NOT generate a fake number).
# The ID number is never in date format (e.g., 1997/12/23 is incorrect).
# Ensure the ID number is exactly 14 digits long.
# Extract text as is without modifying, reformatting, or replacing numbers.
# """
prompt = "Below is the image of one page of a document, as well as some raw textual content that was previously extracted for it. Return the plain text plus the Arabic numbers as: ٠ ١ ٢ ٣ ٤ ٥ ٦ ٧ ٨ ٩. focus on extract accurate numbers specially the zero:٠ representation of this document as if you were reading it naturally. Do not hallucinate."

# image.save(src)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": f"file://{src}"},
            {"type": "text", "text": prompt},
        ],
    }
]
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")
generated_ids = model.generate(**inputs, max_new_tokens=max_tokens)
# generated_ids = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=True, temperature=0.5)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]
# os.remove(src)
print(output_text)


٧٩٤٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.٧٣٣٧.

In [25]:
import json
from transformers import pipeline

def extract_id_number(image_path):
    prompt = """
    Extract the National ID Number from the Egyptian ID card in the image.

    - The National ID is a 14-digit number.
    - It should contain NO spaces, dashes (-), slashes (/), or extra characters.
    - Do NOT return a date format (e.g., 1997/12/23 is incorrect).
    - Ensure the ID number is extracted exactly as it appears on the card.
    - Output the extracted information in JSON format:

    {
        "id_number": "EXTRACTED_14_DIGIT_ID"
    }
    """

    # Load Qari-OCR model
    ocr_pipeline = pipeline("image-to-text", model="NAMAA-Space/Qari-OCR-0.1-VL-2B-Instruct")

    # Run OCR on the image
    extracted_text = ocr_pipeline(image_path)

    # Find the ID number (14 consecutive digits)
    import re
    match = re.search(r"\b\d{14}\b", extracted_text[0]["generated_text"])

    # Return extracted ID number in JSON format
    if match:
        result = {"id_number": match.group()}
    else:
        result = {"error": "No valid 14-digit ID number found."}

    return json.dumps(result, ensure_ascii=False, indent=4)

# Example Usage
image_path = "/content/WhatsApp Image 2025-03-05 at 00.06.19_8e0f6443.jpg"
print(extract_id_number(image_path))


`low_cpu_mem_usage` was None, now default to True since model is quantized.


tokenizer_config.json:   0%|          | 0.00/4.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

Device set to use cuda:0


KeyError: 'input_ids'

In [10]:
import os
import torch
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

# Load model and processor
model_name = "NAMAA-Space/Qari-OCR-0.1-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name, torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained(model_name)

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define input image path
src = "/content/WhatsApp Image 2025-03-05 at 00.06.19_8e0f6443.jpg"
output_txt = os.path.splitext(src)[0] + ".txt"

# Ensure image exists
if not os.path.exists(src):
    raise FileNotFoundError(f"Image not found: {src}")

# Load the image
image = Image.open(src)

# Define prompt
prompt = (
    "Below is the image of one page of a document, as well as some raw textual content "
    "that was previously extracted for it. Just return the plain text representation of "
    "this document as if you were reading it naturally. Do not hallucinate."
)

# Prepare inputs
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": prompt},
        ],
    }
]

# Tokenize input and process vision info
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text], images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt"
).to(device)

# Generate text output
max_tokens = 2000  # Define max tokens
generated_ids = model.generate(**inputs, max_new_tokens=max_tokens)
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]

# Save output to a text file
with open(output_txt, "w", encoding="utf-8") as f:
    f.write(output_text)

print(f"Processed {src} → Saved output to {output_txt}")

# Clear CUDA memory
del inputs, image_inputs, video_inputs, generated_ids, generated_ids_trimmed
torch.cuda.empty_cache()


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 